In [ ]:
from enum import IntEnum

class _dataframe_field(IntEnum):
    original_index = 0
    line_flag = 1
    circle_flag = 2
    arc_flag = 3
    point_flag = 4
    start_x = 5
    start_y = 6
    end_x = 7
    end_y = 8
    mid_x = 9
    mid_y = 10
    angle = 11
    length = perimeter = 12
    d_x = 13
    d_y = 14
    u_x = t_x = 15
    u_y = t_y = 16
    n_x = r_x = 17
    n_y = r_y = 18
    center_x = 19
    center_y = 20
    radius = 21
    start_angle = 22
    end_angle = 23
    arc_span = 24
    
    @classmethod
    def count(cls) -> int: return len(cls)

class _edge_attribute(IntEnum):
    parallel = 0
    offset = 1
    overlap_ratio = 2
    oblique = 3
    intersection_min = 4
    intersection_max = 5
    angle_difference_sin_min = 6
    angle_difference_cos_min = 7
    angle_difference_sin_max = 8
    angle_difference_cos_max = 9
    
    @classmethod
    def count(cls) -> int: return len(cls)

from typing import Tuple
import torch
from torch import Tensor
from torch import newaxis

@staticmethod
def create_obbs(elements:Tensor, width:float, length_extension:float=0.0) -> Tuple[Tensor, Tensor]:
    
    F = _dataframe_field
    n_elements = elements.size(1)
    
    # Filter supported elements
    is_line = elements[F.line_flag] == 1
    is_circle_or_arc = (elements[F.circle_flag] == 1) | (elements[F.arc_flag] == 1)
    
    filter = is_line | is_circle_or_arc
    
    obbs = torch.empty((n_elements, 4, 2), dtype=torch.float32)
    
    if is_line.any():
        # === LINE ELEMENTS ===
        lines = elements[:, is_line]
        
        # Half dimensions
        half_width = width / 2
        half_length = (lines[F.length] + length_extension) / 2
        
        # Compute displacements
        dx_length = lines[F.u_x] * half_length
        dy_length = lines[F.u_y] * half_length
        dx_width = lines[F.n_x] * half_width
        dy_width = lines[F.n_y] * half_width
        
        # Corners (4 per OBB)
        corner1 = torch.stack([lines[F.mid_x] - dx_length - dx_width,
                                lines[F.mid_y] - dy_length - dy_width], dim=1)
        
        corner2 = torch.stack([lines[F.mid_x] + dx_length - dx_width,
                                lines[F.mid_y] + dy_length - dy_width], dim=1)
        
        corner3 = torch.stack([lines[F.mid_x] + dx_length + dx_width,
                                lines[F.mid_y] + dy_length + dy_width], dim=1)
        
        corner4 = torch.stack([lines[F.mid_x] - dx_length + dx_width,
                                lines[F.mid_y] - dy_length + dy_width], dim=1)
        
        # Stack corners
        obbs[is_line] = torch.stack([corner1, corner2, corner3, corner4], dim=1)
    
    if is_circle_or_arc.any():
        # === ARC ELEMENTS ===
        arcs = elements[:, is_circle_or_arc]
        
        margin = width / 2
        
        r_x, r_y, t_x, t_y = arcs[F.r_x], arcs[F.r_y], arcs[F.t_x], arcs[F.t_y]
        arc_span, radius = arcs[F.arc_span], arcs[F.radius]

        dx_length = torch.where(arc_span < torch.pi, t_x * (radius * torch.sin(arc_span/2) + margin), t_x * (radius + margin))
        dy_length = torch.where(arc_span < torch.pi, t_y * (radius * torch.sin(arc_span/2) + margin), t_y * (radius + margin))

        dx_width = r_x * (radius * (1 - torch.cos(arc_span/2)) + margin)
        dy_width = r_y * (radius * (1 - torch.cos(arc_span/2)) + margin)

        dx_margin = r_x * margin
        dy_margin = r_y * margin
        
        corner1 = torch.stack([arcs[F.mid_x] - dx_length + dx_margin,
                                arcs[F.mid_y] - dy_length + dy_margin], dim=1)
        
        corner2 = torch.stack([arcs[F.mid_x] + dx_length + dx_margin,
                                arcs[F.mid_y] + dy_length + dy_margin], dim=1)
        
        corner3 = torch.stack([arcs[F.mid_x] + dx_length - dx_width,
                                arcs[F.mid_y] + dy_length - dy_width], dim=1)
        
        corner4 = torch.stack([arcs[F.mid_x] - dx_length - dx_width,
                                arcs[F.mid_y] - dy_length - dy_width], dim=1)
        
        obbs[is_circle_or_arc] = torch.stack([corner1, corner2, corner3, corner4], dim=1)
    
    elements = elements[:, filter]
    obbs = obbs[filter]
    
    return elements, obbs # Shape obbs (n_elements, 4, 2)

@staticmethod
def find_overlaping_pairs(elements:Tensor, obbs:Tensor) -> Tuple[Tensor, Tensor]:
    
    F = _dataframe_field
    
    # Compute AABBs (Axis-Aligned Bounding Boxes)
    min_x = obbs[..., 0].min(dim=1).values
    max_x = obbs[..., 0].max(dim=1).values
    min_y = obbs[..., 1].min(dim=1).values
    max_y = obbs[..., 1].max(dim=1).values
    
    # Computing AABB intersection matrix
    mask_x = (min_x[:, newaxis] <= max_x[newaxis, :]) & (min_x[newaxis, :] <= max_x[:, newaxis])
    mask_y = (min_y[:, newaxis] <= max_y[newaxis, :]) & (min_y[newaxis, :] <= max_y[:, newaxis])
    aabb_overlap = mask_x & mask_y
    
    # Candidate pairs
    pairs = torch.argwhere(aabb_overlap) # shape (n, 2)
    i, j = pairs[:, 0], pairs[:, 1]
    
    # Remove self-comparisons and duplicates
    mask = (i < j)
    i, j = i[mask], j[mask]
    
    # Get OBBs for pairs
    obbs_i, obbs_j = obbs[i], obbs[j]
    
    # axes per box
    axes_i = torch.stack([elements[F.u_x, i], elements[F.u_y, i], elements[F.n_x, i], elements[F.n_y, i]], dim=1).reshape(-1,2,2)
    axes_j = torch.stack([elements[F.u_x, j], elements[F.u_y, j], elements[F.n_x, j], elements[F.n_y, j]], dim=1).reshape(-1,2,2)
    axes = torch.cat([axes_i, axes_j], axis=1) # shape (n_pairs, 4, 2)
    
    # Project corners onto axes
    projections_i = torch.einsum('nij,nkj->nik', axes, obbs_i) # Shape (n_pairs, 4, 4)
    projections_j = torch.einsum('nij,nkj->nik', axes, obbs_j)
    
    # Interval comparisons on each axis
    min_i = projections_i.min(dim=2).values
    max_i = projections_i.max(dim=2).values
    min_j = projections_j.min(dim=2).values
    max_j = projections_j.max(dim=2).values
    
    separating_axis = (max_i < min_j) | (max_j < min_i) # True if a separating axis exists
    obb_overlap = ~torch.any(separating_axis, dim=1) # True if overlap
    
    return i[obb_overlap], j[obb_overlap]



In [2]:
import ezdxf

dxf_file = "C:\\Users\\Rafael\\Desktop\\parallel_lines.dxf"

doc = ezdxf.readfile(dxf_file)
modelSpace = doc.modelspace()

entities = [entity for entity in modelSpace if entity.dxftype() in ('LINE', 'POINT', 'CIRCLE', 'ARC')]

F = _dataframe_field
dataframe = torch.zeros((F.count(), len(entities)), dtype=torch.float32)

for i, entity in enumerate(entities):
    dataframe[F.original_index,i] = i
    entity_type = entity.dxftype()
    
    if entity_type == 'LINE':
        dataframe[F.line_flag,i] = 1
        dataframe[F.start_x,i], dataframe[F.start_y,i], _ = entity.dxf.start
        dataframe[F.end_x,i], dataframe[F.end_y,i], _ = entity.dxf.end
    
    elif entity_type == 'POINT':
        dataframe[F.point_flag,i] = 1
        dataframe[F.mid_x,i], dataframe[F.mid_y,i], _ = entity.dxf.location
        
    elif entity_type in ('CIRCLE', 'ARC'):
        dataframe[F.circle_flag,i] = 1 if entity_type == 'CIRCLE' else 0
        dataframe[F.arc_flag,i] = 1 if entity_type == 'ARC' else 0
        dataframe[F.radius,i] = entity.dxf.radius
        dataframe[F.center_x,i], dataframe[F.center_y,i], _ = entity.dxf.center
        dataframe[F.start_angle,i] = 0 if entity_type == 'CIRCLE' else entity.dxf.start_angle
        dataframe[F.end_angle,i] = 360 if entity_type == 'CIRCLE' else entity.dxf.end_angle
        
# === LINES ===
is_line = dataframe[F.line_flag] == 1

min_length = 1e-3

dataframe[F.mid_x] = torch.where(is_line, (dataframe[F.start_x] + dataframe[F.end_x]) / 2, dataframe[F.mid_x])
dataframe[F.mid_y] = torch.where(is_line, (dataframe[F.start_y] + dataframe[F.end_y]) / 2, dataframe[F.mid_y])

dataframe[F.d_x] = torch.where(is_line, dataframe[F.end_x] - dataframe[F.start_x], dataframe[F.d_x])
dataframe[F.d_y] = torch.where(is_line, dataframe[F.end_y] - dataframe[F.start_y], dataframe[F.d_y])
dataframe[F.angle] = torch.where(is_line, torch.arctan2(dataframe[F.d_y], dataframe[F.d_x]) % (2*torch.pi), dataframe[F.angle])
dataframe[F.length] = torch.where(is_line, torch.sqrt(dataframe[F.d_x]**2 + dataframe[F.d_y]**2), dataframe[F.length])

is_line &= (dataframe[F.length] > min_length)

# Unit direction and unit perpendicular vectors
dataframe[F.u_x] = torch.where(is_line, dataframe[F.d_x] / dataframe[F.length], dataframe[F.u_x])
dataframe[F.u_y] = torch.where(is_line, dataframe[F.d_y] / dataframe[F.length], dataframe[F.u_y])
dataframe[F.n_x] = torch.where(is_line, -dataframe[F.u_y], dataframe[F.n_x])
dataframe[F.n_y] = torch.where(is_line, dataframe[F.u_x], dataframe[F.n_y])

# === POINTS ===
is_point = dataframe[F.point_flag] == 1

for field in [F.start_x, F.end_x]: dataframe[field] = torch.where(is_point, dataframe[F.mid_x], dataframe[field])
for field in [F.start_y, F.end_y]: dataframe[field] = torch.where(is_point, dataframe[F.mid_y], dataframe[field])

is_line_point = (dataframe[F.line_flag] == 1) & (dataframe[F.length] <= min_length)

dataframe[F.point_flag] = torch.where(is_line_point, 1, dataframe[F.point_flag])
dataframe[F.line_flag] = torch.where(is_line_point, 0, dataframe[F.line_flag])

# === CIRCLES AND ARCS ===
is_circle = dataframe[F.circle_flag] == 1
is_arc = dataframe[F.arc_flag] == 1
is_circle_or_arc = is_circle | is_arc

dataframe[F.start_angle] = torch.where(is_circle_or_arc, torch.deg2rad(dataframe[F.start_angle]), dataframe[F.start_angle])
dataframe[F.end_angle] = torch.where(is_circle_or_arc, torch.deg2rad(dataframe[F.end_angle]), dataframe[F.end_angle])

dataframe[F.arc_span] = torch.where(is_circle, 2 * torch.pi, dataframe[F.arc_span])
dataframe[F.arc_span] = torch.where(is_arc, ((dataframe[F.end_angle] - dataframe[F.start_angle]) % (2*torch.pi)), dataframe[F.arc_span])

mid_angle = torch.where(is_circle_or_arc, (dataframe[F.start_angle] + dataframe[F.arc_span] / 2) % (2*torch.pi), 0)

dataframe[F.start_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(dataframe[F.start_angle]), dataframe[F.start_x])
dataframe[F.start_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(dataframe[F.start_angle]), dataframe[F.start_y])
dataframe[F.end_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(dataframe[F.end_angle]), dataframe[F.end_x])
dataframe[F.end_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(dataframe[F.end_angle]), dataframe[F.end_y])
dataframe[F.mid_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(mid_angle), dataframe[F.mid_x])
dataframe[F.mid_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(mid_angle), dataframe[F.mid_y])

dataframe[F.r_x] = torch.where(is_circle_or_arc, (dataframe[F.mid_x] - dataframe[F.center_x]) / dataframe[F.radius], dataframe[F.r_x])
dataframe[F.r_y] = torch.where(is_circle_or_arc, (dataframe[F.mid_y] - dataframe[F.center_y]) / dataframe[F.radius], dataframe[F.r_y])
dataframe[F.t_x] = torch.where(is_circle_or_arc, -dataframe[F.r_y], dataframe[F.t_x])
dataframe[F.t_y] = torch.where(is_circle_or_arc, dataframe[F.r_x], dataframe[F.t_y])

dataframe[F.perimeter] = torch.where(is_circle_or_arc, dataframe[F.radius] * dataframe[F.arc_span], dataframe[F.perimeter])

In [3]:
# Parameters
line_obb_width=0.5
parallel_max_offset=25.
parallel_angle_tolerance=0.1

max_offset=None; angle_tolerance=None

In [4]:
max_offset = max_offset or parallel_max_offset
angle_tolerance = (angle_tolerance or parallel_angle_tolerance) * torch.pi/180

# Filter valid line indices
is_line = dataframe[F.line_flag] == 1
lines = dataframe[:, is_line]

# Compute OBBs
lines, obbs = create_obbs(elements=lines, width=max_offset, length_extension=line_obb_width) # Shape obbs (n_lines, 4, 2)

# Get the pairs of overlapping obbs
i, j = find_overlaping_pairs(lines, obbs)
lines_a, lines_b = lines[:, i], lines[:, j]

# Compute absolute angle difference in [0, pi]
angle_difference_b_a = (lines_b[F.angle] - lines_a[F.angle]) % (2*torch.pi)
min_angle_difference = angle_difference_b_a % torch.pi
min_angle_difference = torch.minimum(min_angle_difference, torch.pi - min_angle_difference)

# Keep only pairs with angle difference below threshold
parallel = min_angle_difference <= angle_tolerance
lines_a, lines_b = lines_a[:, parallel], lines_b[:, parallel]
angle_difference_b_a = angle_difference_b_a[parallel]

# get the angle difference sin and cos
angle_difference_b_a_sin = torch.sin(angle_difference_b_a)
angle_difference_b_a_cos = torch.cos(angle_difference_b_a)

In [ ]:
# Project B's endpoints onto A
t1 = (lines_b[F.start_x] - lines_a[F.start_x]) * lines_a[F.u_x] + \
     (lines_b[F.start_y] - lines_a[F.start_y]) * lines_a[F.u_y]

t2 = (lines_b[F.end_x] - lines_a[F.start_x]) * lines_a[F.u_x] + \
     (lines_b[F.end_y] - lines_a[F.start_y]) * lines_a[F.u_y]

# Get the projected range
tmin, tmax = torch.minimum(t1, t2), torch.maximum(t1, t2)

# Clip the projection range within [0, length_a]
overlap_start = torch.clamp(tmin, min=0)
overlap_end = torch.clamp(tmax, max=lines_a[F.length])
overlap_mid = (overlap_start + overlap_end) / 2

# Overlap ratio
overlap_length = torch.clamp(overlap_end - overlap_start, min=0)
overlap_a_b = overlap_length / lines_a[F.length]
overlap_b_a = overlap_length / lines_b[F.length]

# Midpoint of projected overlap on line A
overlap_mid_xa = lines_a[F.start_x] + lines_a[F.u_x] * overlap_mid
overlap_mid_ya = lines_a[F.start_y] + lines_a[F.u_y] * overlap_mid

# Perpendicular distance to line B
dx = overlap_mid_xa - lines_b[F.mid_x]
dy = overlap_mid_ya - lines_b[F.mid_y]

# Perpendicular distances relative to overlaping segment midpoint
distance_a_b = dx * lines_b[F.n_x] + dy * lines_b[F.n_y]
distance_b_a = -dx * lines_a[F.n_x] - dy * lines_a[F.n_y]

In [6]:
distance_a_b = dx * lines_b[F.n_x] + dy * lines_b[F.n_y]
distance_a_b

tensor([ 19.9665, -19.8031])

In [7]:
distance_b_a = -dx * lines_a[F.n_x] - dy * lines_a[F.n_y]
distance_b_a

tensor([-19.9665, -19.8031])